In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

/opt/conda/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [34]:
df = pd.read_csv('UCI_Credit_Card.csv')

In [35]:
df = df.rename(columns={'default.payment.next.month': 'def_pay', 
                        'PAY_0': 'PAY_1'})

# Decision tree

In [3]:
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
from xgboost import XGBClassifier

In [37]:
y = df['def_pay'].copy()

In [38]:
features = ['LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_1', 'PAY_2',
       'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']
X = df[features].copy()

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [40]:
fil = (df.EDUCATION == 5) | (df.EDUCATION == 6) | (df.EDUCATION == 0)
df.loc[fil, 'EDUCATION'] = 4

In [41]:
df.loc[df.MARRIAGE == 0, 'MARRIAGE'] = 3

In [42]:
fil = (df.PAY_1 == -2) | (df.PAY_1 == -1) | (df.PAY_1 == 0)
df.loc[fil, 'PAY_1'] = 0
fil = (df.PAY_2 == -2) | (df.PAY_2 == -1) | (df.PAY_2 == 0)
df.loc[fil, 'PAY_2'] = 0
fil = (df.PAY_3 == -2) | (df.PAY_3 == -1) | (df.PAY_3 == 0)
df.loc[fil, 'PAY_3'] = 0
fil = (df.PAY_4 == -2) | (df.PAY_4 == -1) | (df.PAY_4 == 0)
df.loc[fil, 'PAY_4'] = 0
fil = (df.PAY_5 == -2) | (df.PAY_5 == -1) | (df.PAY_5 == 0)
df.loc[fil, 'PAY_5'] = 0
fil = (df.PAY_6 == -2) | (df.PAY_6 == -1) | (df.PAY_6 == 0)
df.loc[fil, 'PAY_6'] = 0

In [16]:
df.loc[df.PAY_1 > 0, 'PAY_1']

0        2
13       1
15       1
18       1
19       1
        ..
29981    1
29991    3
29994    2
29997    4
29998    1
Name: PAY_1, Length: 6818, dtype: int64

In [17]:
X_train[X_train.columns[:16]]

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5
21753,80000.0,2,2,2,24,0,0,0,0,0,0,75125.0,77353.0,78321.0,73731.0,39643.0
251,30000.0,1,2,2,28,0,0,0,0,0,0,29242.0,29507.0,29155.0,25255.0,22001.0
22941,180000.0,2,5,1,44,0,0,-1,-1,-1,-1,20916.0,0.0,850.0,0.0,6881.0
618,60000.0,1,1,2,25,0,0,0,0,0,0,58839.0,53235.0,38533.0,39639.0,39619.0
17090,130000.0,2,2,2,25,0,0,0,0,0,0,111587.0,112348.0,114734.0,117823.0,120854.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29802,50000.0,1,2,2,32,0,0,0,0,0,0,52475.0,53600.0,55739.0,55957.0,29238.0
5390,200000.0,1,1,2,37,2,2,2,2,2,2,157131.0,166590.0,168386.0,164182.0,169029.0
860,50000.0,1,1,2,26,-2,-2,-2,-2,-2,-2,0.0,0.0,0.0,0.0,0.0
15795,70000.0,2,2,2,25,0,0,0,0,2,2,73939.0,70488.0,51152.0,35122.0,28633.0


In [18]:
len(y_train)

24000

In [19]:
y_train.sum()

5323

In [43]:
df['SE_MA_2'] = 0
df.loc[((df.SEX == 1) & (df.MARRIAGE == 1)) , 'SE_MA_2'] = 1 #married man
df.loc[((df.SEX == 1) & (df.MARRIAGE == 2)) , 'SE_MA_2'] = 2 #single man
df.loc[((df.SEX == 1) & (df.MARRIAGE == 3)) , 'SE_MA_2'] = 3 #divorced man
df.loc[((df.SEX == 2) & (df.MARRIAGE == 1)) , 'SE_MA_2'] = 4 #married woman
df.loc[((df.SEX == 2) & (df.MARRIAGE == 2)) , 'SE_MA_2'] = 5 #single woman
df.loc[((df.SEX == 2) & (df.MARRIAGE == 3)) , 'SE_MA_2'] = 6 #divorced woman

In [21]:
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,def_pay,SE_MA_2
0,1,20000.0,2,2,1,24,2,2,0,0,...,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1,4
1,2,120000.0,2,2,2,26,0,2,0,0,...,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1,5
2,3,90000.0,2,2,2,34,0,0,0,0,...,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0,5
3,4,50000.0,2,2,1,37,0,0,0,0,...,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0,4
4,5,50000.0,1,2,1,57,0,0,0,0,...,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0,1


In [22]:
df.AGE

0        24
1        26
2        34
3        37
4        57
         ..
29995    39
29996    43
29997    37
29998    41
29999    46
Name: AGE, Length: 30000, dtype: int64

In [23]:
df.def_pay.value_counts()

0    23364
1     6636
Name: def_pay, dtype: int64

In [31]:
len(test_index)

3000

In [140]:
train_data, test_data = train_test_split(df, test_size=0.1)

In [146]:
train_data.to_csv('train.csv', index=0)
test_data.to_csv('test.csv', index=0)

In [ ]:
AdaBoostClassifier(DecisionTreeClassifier(max_depth = 3, class_weight = "balanced"),
                               n_estimators = 100), 

In [ ]:
DecisionTreeClassifier(n_es)

In [83]:
best_clf = []
param_grid = {
              'criterion': ['entropy'],
              'class_weight' : ['balanced'],
              'max_depth': [3, 4, 5]}

acc_scorer = make_scorer(f1_score)
decision_forest = GridSearchCV(DecisionTreeClassifier(), param_grid, scoring=acc_scorer, cv=3)
decision_forest.fit(X_train, y_train)
print(decision_forest.best_estimator_)
print(decision_forest.best_score_)
best_clf.append(decision_forest.best_estimator_)

DecisionTreeClassifier(class_weight='balanced', criterion='entropy',
                       max_depth=5)
0.524917732840431


In [84]:
param_grid = {'n_estimators': [300, 400, 500], # It is going to be a long search
              'criterion': ['entropy'],
              'class_weight' : ['balanced'],
              'max_depth': [3, 4, 5]}

acc_scorer = make_scorer(f1_score)
random_forest = GridSearchCV(RandomForestClassifier(), param_grid, scoring=acc_scorer, cv=3)
random_forest.fit(X_train, y_train)
print(random_forest.best_estimator_)
print(random_forest.best_score_)
best_clf.append(random_forest.best_estimator_)

RandomForestClassifier(class_weight='balanced', criterion='entropy',
                       max_depth=5, n_estimators=300)
0.5452740323229858


In [92]:
param_grid = {'n_estimators': [200, 300, 400],
              'max_depth': [3, 4, 5]}

acc_scorer = make_scorer(f1_score)
gb_forest = GridSearchCV(GradientBoostingClassifier(), param_grid, scoring=acc_scorer, cv=3)
gb_forest.fit(X_train, y_train)
print(gb_forest.best_estimator_)
print(gb_forest.best_score_)
best_clf.append(gb_forest.best_estimator_)

GradientBoostingClassifier(n_estimators=200)
0.4789395643103946


In [94]:
param_grid = {'n_estimators': [300, 400, 500], # It is going to be a long search
              'max_depth': [3, 4, 5]}

acc_scorer = make_scorer(f1_score)
xgb_forest = GridSearchCV(XGBClassifier(), param_grid, scoring=acc_scorer, cv=3)
xgb_forest.fit(X_train, y_train)
print(xgb_forest.best_estimator_)
print(xgb_forest.best_score_)
best_clf.append(xgb_forest.best_estimator_)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=3, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=300,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0, ...)
0.47533923197552236


In [95]:
best_clf

[DecisionTreeClassifier(class_weight='balanced', criterion='entropy',
                        max_depth=5),
 RandomForestClassifier(class_weight='balanced', criterion='entropy',
                        max_depth=5, n_estimators=300),
 GradientBoostingClassifier(n_estimators=15),
 GradientBoostingClassifier(n_estimators=200),
 XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
               colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
               early_stopping_rounds=None, enable_categorical=False,
               eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
               grow_policy='depthwise', importance_type=None,
               interaction_constraints='', learning_rate=0.300000012,
               max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
               max_delta_step=0, max_depth=3, max_leaves=0, min_child_weight=1,
               missing=nan, monotone_constraints='()', n_estimators=300,
               n_jobs=0, num_paral

In [101]:
ada_clf = AdaBoostClassifier(random_forest.best_estimator_, )
ada_clf.fit(X_train, y_train)
pred = ada_clf.predict(X_test)
score = f1_score(y_true=y_test, y_pred=pred)

In [102]:
score

0.5278058645096056

In [105]:
# rf_clf = random_forest.best_estimator_
# rf_clf.fit(X_train, y_train)
pred = rf_clf.predict(X_test)
score = f1_score(y_true=y_test, y_pred=pred)
print(score)

0.5351921115266917


In [109]:
param_grid = {'n_estimators': [10, 20, 30]}

acc_scorer = make_scorer(f1_score)
ad_forest = GridSearchCV(AdaBoostClassifier(random_forest.best_estimator_), param_grid, scoring=acc_scorer, cv=3)
ad_forest.fit(X_train, y_train)
print(ada_forest.best_estimator_)
print(ada_forest.best_score_)
best_clf.append(ada_forest.best_estimator_)

NameError: name 'ada_forest' is not defined

In [110]:
print(ad_forest.best_estimator_)
print(ad_forest.best_score_)
best_clf.append(ad_forest.best_estimator_)

AdaBoostClassifier(base_estimator=RandomForestClassifier(class_weight='balanced',
                                                         criterion='entropy',
                                                         max_depth=5,
                                                         n_estimators=300),
                   n_estimators=10)
0.5428567697573968


In [114]:
features = ['ID', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_1',
       'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6', 'SE_MA_2']

In [117]:
df

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,def_pay,SE_MA_2
0,1,20000.0,2,2,1,24,2,2,0,0,...,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1,4
1,2,120000.0,2,2,2,26,0,2,0,0,...,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1,5
2,3,90000.0,2,2,2,34,0,0,0,0,...,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0,5
3,4,50000.0,2,2,1,37,0,0,0,0,...,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0,4
4,5,50000.0,1,2,1,57,0,0,0,0,...,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,29996,220000.0,1,3,1,39,0,0,0,0,...,31237.0,15980.0,8500.0,20000.0,5003.0,3047.0,5000.0,1000.0,0,1
29996,29997,150000.0,1,3,2,43,0,0,0,0,...,5190.0,0.0,1837.0,3526.0,8998.0,129.0,0.0,0.0,0,2
29997,29998,30000.0,1,2,2,37,4,3,2,0,...,20582.0,19357.0,0.0,0.0,22000.0,4200.0,2000.0,3100.0,1,2
29998,29999,80000.0,1,3,1,41,1,0,0,0,...,11855.0,48944.0,85900.0,3409.0,1178.0,1926.0,52964.0,1804.0,1,1


In [118]:
train_index

array([    1,     2,     3, ..., 29996, 29997, 29999])

In [116]:
kf = KFold(n_splits=10, shuffle=True)

mdl = []
fold = []
scr = []

for i, (train_index, test_index) in enumerate(kf.split(df)):
    training = df.iloc[train_index:]
    valid = df.iloc[test_index:]
    print(i)
    model = ad_forest.best_estimator_
    feats = training[features]
    label = training['def_pay']
    valid_feats = valid[features]
    valid_label = valid['def_pay']
    model.fit(feats, label) 
    pred = model.predict(valid_feats)
    score = f1_score(y_true=valid_label, y_pred=pred)
    fold.append(i+1)
    scr.append(score)
    mdl.append(model)
    print(f"{i+1} score: {score}")
    

performance = pd.DataFrame({'Model': mdl, 'Score':scr,'Fold':fold})
g_normal = ggplot(performance,aes(x='Fold',y='Score',group = 'Model',color = 'Model')) + geom_point() + geom_line()
print(g_normal)

TypeError: cannot do positional indexing on RangeIndex with these indexers [[    1     2     3 ... 29996 29997 29999]] of type ndarray

# NN

In [4]:
import torch.nn.functional as F
import torch.nn as nn
import torch
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from torchmetrics import F1Score

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import wandb

/opt/conda/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [5]:
device = torch.device('cuda')

In [6]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [7]:
X_train = train_df[['LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_1',
       'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6', 'SE_MA_2']]
y_train = train_df['def_pay']

X_test = test_df[['LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_1',
       'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6', 'SE_MA_2']]
y_test = test_df['def_pay']

In [8]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=6000, random_state=42)

In [9]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [18]:
class TrainData(Dataset):
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = torch.tensor(y_data, dtype=torch.int64)
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
    
    def __len__ (self):
        return len(self.X_data)
    
class ValidData(Dataset):
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = torch.tensor(y_data, dtype=torch.int64)
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
    
    def __len__ (self):
        return len(self.X_data)


## test data    
class TestData(Dataset):
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = torch.tensor(y_data, dtype=torch.int64)
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
    
    def __len__ (self):
        return len(self.X_data)

In [20]:
train_data = TrainData(torch.FloatTensor(X_train), torch.FloatTensor(y_train.values))
valid_data = TrainData(torch.FloatTensor(X_valid), torch.FloatTensor(y_valid.values))
test_data = TestData(torch.FloatTensor(X_test), torch.FloatTensor(y_valid.values))

<ipython-input-18-5afdea0073af>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.y_data = torch.tensor(y_data, dtype=torch.int64)
<ipython-input-18-5afdea0073af>:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.y_data = torch.tensor(y_data, dtype=torch.int64)


In [21]:
train_loader = DataLoader(dataset=train_data, batch_size=32, shuffle=True)
valid_loader = DataLoader(dataset=train_data, batch_size=32)
test_loader = DataLoader(dataset=test_data, batch_size=32)

In [22]:
class BinaryClassification(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(24, 32)
        self.dropout1 = nn.Dropout(0.2)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(32, 16)
        self.dropout2 = nn.Dropout(0.2)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(16, 8)
        self.dropout3 = nn.Dropout(0.2)
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(8, 2)
        self.dropout4 = nn.Dropout(0.2)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # print('0: ', x.size())
        x = self.fc1(x)
        x = self.dropout1(x)
        x = self.relu1(x)
        # print('1: ', x.size())
        x = self.fc2(x)
        x = self.dropout2(x)
        x = self.relu2(x)
        # print('2: ', x.size())
        x = self.fc3(x)
        x = self.dropout3(x)
        x = self.relu3(x)
        # print('3: ', x.size())
        x = self.fc4(x)
        x = self.dropout4(x)
        x = self.sigmoid(x)
        
        return x

In [23]:
class FocalLoss(nn.Module):
    def __init__(self, gamma=0, alpha=None, size_average=True, device='cuda'):
        super(FocalLoss, self).__init__()
        """
        gamma(int) : focusing parameter.
        alpha(list) : alpha-balanced term.
        size_average(bool) : whether to apply reduction to the output.
        """
        self.gamma = gamma
        self.alpha = alpha
        self.size_average = size_average
        self.device = device

    def forward(self, pred, target):
        # input : N * C (btach_size, num_class)
        # target : N (batch_size)

        CE = F.cross_entropy(pred, target, reduction='none')  # -log(pt)
        pt = torch.exp(-CE)  # pt
        loss = (1 - pt) ** self.gamma * CE  # -(1-pt)^rlog(pt)

        if self.alpha is not None:
            alpha = torch.tensor(self.alpha, dtype=torch.float).to(self.device)
            # in case that a minority class is not selected when mini-batch sampling
            if len(self.alpha) != len(torch.unique(target)):
                temp = torch.zeros(len(self.alpha)).to(self.device)
                temp[torch.unique(target)] = alpha.index_select(0, torch.unique(target))
                alpha_t = temp.gather(0, target)
                loss = alpha_t * loss
            else:
                alpha_t = alpha.gather(0, target)
                loss = alpha_t * loss

        if self.size_average:
            loss = torch.mean(loss)

        return loss

In [31]:
model = BinaryClassification()
model.to(device)
criterion = FocalLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.00001)

In [32]:
wandb.init(project='test4')

train_f1_score,███████████████████▇▂▁▁▅█████████████
train_loss,▆▆▆▆▅▅▅▄▄▄▃▃▃▂▂▂▁▁▁▇███▇▇▇▆▆▅▅▄▄▃▃▂▂▂
valid_f1_score,████████████████████▁▁▁▇█████████████
valid_loss,▆▆▆▅▅▅▅▄▄▃▃▃▂▂▂▂▁▁▁▇███▇▇▇▆▆▅▅▄▄▃▃▂▂▂
train_f1_score,511.5
train_loss,358.35027
valid_f1_score,511.28125
valid_loss,355.87038


In [33]:
model.train()
wandb.run.name = 'test3'
f1 = F1Score(num_classes=2).to(device)
# tqdm
for e in range(1, 20):
    train_loss = 0
    train_acc = 0
    valid_loss = 0
    valid_acc = 0
    
    for X_batch, y_batch in train_loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)
        optimizer.zero_grad()
        
        y_pred = model(X_batch)
        
        loss = criterion(y_pred, y_batch)
        acc = f1(torch.argmax(y_pred, dim=1), y_batch)
        
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        train_acc += acc.item()
        
    for X_batch, y_batch in valid_loader:
        model.eval()
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)
        
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        acc = f1(torch.argmax(y_pred, dim=1), y_batch)
        valid_loss += loss.item()
        valid_acc += acc.item()
        
    wandb.log({"train_loss": train_loss, "train_f1_score": train_acc,
              "valid_loss": valid_loss, "valid_f1_score": valid_acc})

test_acc = 0
for X_batch, y_batch in test_loader:
    model.eval()
    X_batch = X_batch.to(device)
    y_batch = y_batch.to(device)

    y_pred = model(X_batch)
    acc = f1(torch.argmax(y_pred, dim=1), y_batch)
    test_acc += acc.item()
wandb.log({"test_f1_score": test_acc})

In [34]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }